<a href="https://colab.research.google.com/github/MoralesTorres/ML_Notebooks/blob/master/EEG_Psychiatric_Disorders_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

DATA_PATH = "/content/EEG.machinelearing_data_BRMH.csv.zip"

# Reads the first CSV inside the zip automatically
df = pd.read_csv(DATA_PATH, compression="zip")

display(df.head())
print("Shape:", df.shape)
print("\nDtypes count:\n", df.dtypes.value_counts())
print("\nMissing values (top 20):\n", df.isna().sum().sort_values(ascending=False).head(20))


,no.,sex,age,eeg.date,education,IQ,main.disorder,specific.disorder,AB.A.delta.a.FP1,AB.A.delta.b.FP2,...,COH.F.gamma.o.Pz.p.P4,COH.F.gamma.o.Pz.q.T6,COH.F.gamma.o.Pz.r.O1,COH.F.gamma.o.Pz.s.O2,COH.F.gamma.p.P4.q.T6,COH.F.gamma.p.P4.r.O1,COH.F.gamma.p.P4.s.O2,COH.F.gamma.q.T6.r.O1,COH.F.gamma.q.T6.s.O2,COH.F.gamma.r.O1.s.O2
0,1,M,57.0,2012.8.30,NaN,NaN,Addictive disorder,Alcohol use disorder,35.998557,21.717375,...,55.989192,16.739679,23.452271,45.678820,30.167520,16.918761,48.850427,9.422630,34.507082,28.613029
1,2,M,37.0,2012.9.6,6.0,120.0,Addictive disorder,Alcohol use disorder,13.425118,11.002916,...,45.595619,17.510824,26.777368,28.201062,57.108861,32.375401,60.351749,13.900981,57.831848,43.463261
2,3,M,32.0,2012.9.10,16.0,113.0,Addictive disorder,Alcohol use disorder,29.941780,27.544684,...,99.475453,70.654171,39.131547,69.920996,71.063644,38.534505,69.908764,27.180532,64.803155,31.485799
3,4,M,35.0,2012.10.8,18.0,126.0,Addictive disorder,Alcohol use disorder,21.496226,21.846832,...,59.986561,63.822201,36.478254,47.117006,84.658376,24.724096,50.299349,35.319695,79.822944,41.141873
4,5,M,36.0,2012.10.18,16.0,112.0,Addictive disorder,Alcohol use disorder,37.775667,33.607679,...,61.462720,59.166097,51.465531,58.635415,80.685608,62.138436,75.888749,61.003944,87.455509,70.531662


Shape: (945, 1149)

Dtypes count:
 float64    1144
object        4
int64         1
Name: count, dtype: int64

Missing values (top 20):
 Unnamed: 122            945
education                15
IQ                       13
COH.D.beta.j.Cz.s.O2      0
COH.D.beta.j.Cz.r.O1      0
COH.D.beta.j.Cz.q.T6      0
COH.D.beta.j.Cz.p.P4      0
COH.D.beta.j.Cz.o.Pz      0
COH.D.beta.j.Cz.n.P3      0
COH.D.beta.j.Cz.m.T5      0
COH.D.beta.j.Cz.l.T4      0
COH.D.beta.k.C4.l.T4      0
COH.D.beta.i.C3.s.O2      0
COH.D.beta.i.C3.r.O1      0
COH.D.beta.i.C3.q.T6      0
COH.D.beta.i.C3.p.P4      0
COH.D.beta.i.C3.o.Pz      0
COH.D.beta.i.C3.n.P3      0
COH.D.beta.i.C3.m.T5      0
COH.D.beta.j.Cz.k.C4      0
dtype: int64


In [2]:
import pandas as pd
import numpy as np

DATA_PATH = "/content/EEG.machinelearing_data_BRMH.csv.zip"
df = pd.read_csv(DATA_PATH, compression="zip")

print("Raw shape:", df.shape)

# Drop Unnamed:* and all-null columns
unnamed = [c for c in df.columns if c.lower().startswith("unnamed")]
df = df.drop(columns=unnamed, errors="ignore")
df = df.dropna(axis=1, how="all")

print("Clean shape:", df.shape)
print("Top missing:\n", df.isna().sum().sort_values(ascending=False).head(10))


Raw shape: (945, 1149)
Clean shape: (945, 1148)
Top missing:
 education               15
IQ                      13
COH.D.beta.j.Cz.s.O2     0
COH.D.beta.j.Cz.r.O1     0
COH.D.beta.j.Cz.q.T6     0
COH.D.beta.j.Cz.p.P4     0
COH.D.beta.j.Cz.o.Pz     0
COH.D.beta.j.Cz.n.P3     0
COH.D.beta.j.Cz.m.T5     0
COH.D.beta.j.Cz.l.T4     0
dtype: int64


In [3]:
# Ayuda rápida para encontrar columnas de diagnóstico
diag_cols = [c for c in df.columns if "disorder" in c.lower() or "diagn" in c.lower() or "label" in c.lower()]
print("Diagnosis-related columns:", diag_cols)


Diagnosis-related columns: ['main.disorder', 'specific.disorder']


In [4]:
# EJEMPLO: ajusta según tus categorías reales
# Supongamos que tienes 'specific.disorder' y 'main.disorder'
specific_col = "specific.disorder"
main_col = "main.disorder"

# Normaliza strings
df[specific_col] = df[specific_col].astype(str).str.lower()
df[main_col] = df[main_col].astype(str).str.lower()

def map_3_classes(row):
    spec = row[specific_col]
    main = row[main_col]

    is_aud = ("aud" in spec) or ("alcohol" in spec) or ("addictive" in main)
    is_dep = ("depress" in main) or ("mdd" in spec)

    if is_aud and is_dep:
        return "AUD+DEP"
    elif is_aud:
        return "AUD"
    else:
        return "NO_AUD"

df["y_3clases"] = df.apply(map_3_classes, axis=1)
print(df["y_3clases"].value_counts())


y_3clases
NO_AUD    759
AUD       186
Name: count, dtype: int64


In [5]:
# Ejemplo: flags de missing y transformaciones en demográficas si existen
for col in ["IQ", "education", "age"]:
    if col in df.columns:
        df[f"{col}_missing"] = df[col].isna().astype(int)

# Transformaciones robustas (si existen)
if "IQ" in df.columns:
    df["IQ_log1p"] = np.log1p(df["IQ"].fillna(df["IQ"].median()))

if "education" in df.columns:
    df["education_log1p"] = np.log1p(df["education"].fillna(df["education"].median()))

# Binning (discretización) — opcional y justificable
if "age" in df.columns:
    df["age_bin"] = pd.cut(df["age"], bins=[0, 18, 25, 35, 50, 120],
                           labels=["<18","18-25","26-35","36-50","50+"])


In [6]:
import re

# Detecta columnas COH (coherencia) y agrupa por banda si el nombre lo trae
coh_cols = [c for c in df.columns if c.lower().startswith("coh")]
print("COH cols:", len(coh_cols))

bands = ["delta","theta","alpha","beta","highbeta","gamma"]
for b in bands:
    cols_b = [c for c in coh_cols if re.search(rf"\.{b}\.", c.lower())]
    if cols_b:
        df[f"COH_{b}_mean"] = df[cols_b].mean(axis=1)
        df[f"COH_{b}_std"]  = df[cols_b].std(axis=1)
        df[f"COH_{b}_median"] = df[cols_b].median(axis=1)

# Si hay otra familia (por ejemplo AB.*), puedes hacer lo mismo:
ab_cols = [c for c in df.columns if c.lower().startswith("ab.")]
print("AB cols:", len(ab_cols))
for b in bands:
    cols_b = [c for c in ab_cols if re.search(rf"\.{b}\.", c.lower())]
    if cols_b:
        df[f"AB_{b}_mean"] = df[cols_b].mean(axis=1)
        df[f"AB_{b}_std"]  = df[cols_b].std(axis=1)


COH cols: 1026
AB cols: 114


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PowerTransformer
from sklearn.impute import SimpleImputer

target = "y_3clases"  # o la etiqueta que decidan
y = df[target]

X = df.drop(columns=[target])

# Detectar numéricas y categóricas
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()

numeric_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("power", PowerTransformer(method="yeo-johnson")),  # estabiliza distribuciones (sirve si hay sesgo)
    ("scaler", StandardScaler())
])

categorical_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, num_cols),
        ("cat", categorical_pipe, cat_cols)
    ],
    remainder="drop"
)


In [8]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline

# VarianceThreshold va después de preprocess (porque one-hot y scaling ya están)
var_filter = Pipeline(steps=[
    ("preprocess", preprocess),
    ("var", VarianceThreshold(threshold=0.0))  # quita constantes
])


In [9]:
# Filtrado por correlación alta en columnas numéricas originales (rápido y justificable)
corr = df[num_cols].corr().abs()
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
to_drop = [col for col in upper.columns if any(upper[col] > 0.95)]
print("Highly correlated numeric cols to drop:", len(to_drop))


Highly correlated numeric cols to drop: 219


In [10]:
from sklearn.feature_selection import SelectKBest, f_classif

anova_pipe = Pipeline(steps=[
    ("preprocess", preprocess),
    ("select", SelectKBest(score_func=f_classif, k=200))  # ajusta k
])


In [11]:
from sklearn.decomposition import PCA

pca_pipe = Pipeline(steps=[
    ("preprocess", preprocess),
    ("pca", PCA(n_components=0.95, random_state=42))
])


In [12]:
from sklearn.decomposition import FactorAnalysis

fa_pipe = Pipeline(steps=[
    ("preprocess", preprocess),
    ("fa", FactorAnalysis(n_components=50, random_state=42))
])


In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import StratifiedKFold, cross_val_score

clf = LogisticRegression(max_iter=5000, class_weight="balanced")

# Comparar 2 pipelines: ANOVA vs PCA (macro-F1)
pipe_anova_model = Pipeline(steps=[
    ("anova", anova_pipe),
    ("clf", clf)
])

pipe_pca_model = Pipeline(steps=[
    ("pca", pca_pipe),
    ("clf", clf)
])

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scores_anova = cross_val_score(pipe_anova_model, X, y, cv=cv, scoring="f1_macro")
scores_pca = cross_val_score(pipe_pca_model, X, y, cv=cv, scoring="f1_macro")

print("ANOVA macro-F1:", scores_anova.mean(), "+/-", scores_anova.std())
print("PCA   macro-F1:", scores_pca.mean(), "+/-", scores_pca.std())


/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [1146 1793] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [1146 1790] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.12/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw
/usr/local/lib/python3.12/dist-packages/sklearn/featur

ANOVA macro-F1: 1.0 +/- 0.0
PCA   macro-F1: 0.6736805839630593 +/- 0.02420513672612031


In [14]:
sus = [c for c in df.columns if any(k in c.lower() for k in ["disorder","diagn","label","class","target","group"])]
sus


['main.disorder', 'specific.disorder']

In [15]:
target = "y_3clases"

# columnas que NO deben entrar a X (incluye las usadas para crear y)
drop_cols = [target] + sus

X = df.drop(columns=drop_cols, errors="ignore")
y = df[target]
print("X shape:", X.shape, "y:", y.shape)


X shape: (945, 1182) y: (945,)


In [16]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.impute import SimpleImputer
import numpy as np

num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()

numeric_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("power", PowerTransformer(method="yeo-johnson")),
    ("scaler", StandardScaler())
])

from sklearn.preprocessing import OneHotEncoder
categorical_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer([
    ("num", numeric_pipe, num_cols),
    ("cat", categorical_pipe, cat_cols)
])

anova_pipe = Pipeline([
    ("preprocess", preprocess),
    ("var", VarianceThreshold(threshold=0.0)),
    ("select", SelectKBest(score_func=f_classif, k=200)),
])


In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score

clf = LogisticRegression(max_iter=5000, class_weight="balanced")
pipe = Pipeline([("fe", anova_pipe), ("clf", clf)])

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(pipe, X, y, cv=cv, scoring="f1_macro")
print(scores.mean(), scores.std())


0.6875813237487846 0.022717337358314566


In [18]:
# Ajustando estos nombres si existen en el universo
for c in ["main.disorder", "specific.disorder"]:
    if c in df.columns:
        print("\n", c)
        print(df[c].astype(str).str.lower().value_counts().head(20))



 main.disorder
main.disorder
mood disorder                         266
addictive disorder                    186
trauma and stress related disorder    128
schizophrenia                         117
anxiety disorder                      107
healthy control                        95
obsessive compulsive disorder          46
Name: count, dtype: int64

 specific.disorder
specific.disorder
depressive disorder               199
schizophrenia                     117
healthy control                    95
alcohol use disorder               93
behavioral addiction disorder      93
bipolar disorder                   67
panic disorder                     59
posttraumatic stress disorder      52
social anxiety disorder            48
obsessive compulsitve disorder     46
acute stress disorder              38
adjustment disorder                38
Name: count, dtype: int64


In [19]:
# --- 3 clases desde specific.disorder ---
df["specific.disorder"] = df["specific.disorder"].astype(str).str.lower().str.strip()

map3 = {
    "healthy control": "HEALTHY",
    "alcohol use disorder": "AUD",
    "depressive disorder": "DEP",
}

df["y_3clases"] = df["specific.disorder"].map(map3)

# quedarnos SOLO con las 3 clases objetivo
df3 = df[df["y_3clases"].notna()].copy()
print(df3["y_3clases"].value_counts())
print("df3 shape:", df3.shape)


y_3clases
DEP        199
HEALTHY     95
AUD         93
Name: count, dtype: int64
df3 shape: (387, 1185)


In [20]:
target = "y_3clases"

# Saca cualquier columna de diagnóstico/etiqueta para evitar leakage
drop_like = ["disorder", "diagn", "label", "target", "class", "group"]
sus = [c for c in df3.columns if any(k in c.lower() for k in drop_like)]

X = df3.drop(columns=[target] + sus, errors="ignore")
y = df3[target]

print("Dropped cols:", sus)
print("X shape:", X.shape, "y shape:", y.shape)


Dropped cols: ['main.disorder', 'specific.disorder']
X shape: (387, 1182) y shape: (387,)


In [21]:
import re
import numpy as np

df3 = df3.copy()

# Agregados por banda para COH y AB
bands = ["delta","theta","alpha","beta","highbeta","gamma"]

coh_cols = [c for c in df3.columns if c.lower().startswith("coh.")]
ab_cols  = [c for c in df3.columns if c.lower().startswith("ab.")]

print("COH cols:", len(coh_cols), "AB cols:", len(ab_cols))

for b in bands:
    # COH
    cols_b = [c for c in coh_cols if re.search(rf"\.{b}\.", c.lower())]
    if cols_b:
        df3[f"COH_{b}_mean"]   = df3[cols_b].mean(axis=1)
        df3[f"COH_{b}_std"]    = df3[cols_b].std(axis=1)
        df3[f"COH_{b}_median"] = df3[cols_b].median(axis=1)

    # AB
    cols_b = [c for c in ab_cols if re.search(rf"\.{b}\.", c.lower())]
    if cols_b:
        df3[f"AB_{b}_mean"] = df3[cols_b].mean(axis=1)
        df3[f"AB_{b}_std"]  = df3[cols_b].std(axis=1)

print("After FE shape:", df3.shape)


COH cols: 1026 AB cols: 114
After FE shape: (387, 1185)


In [22]:
target = "y_3clases"
drop_cols = ["main.disorder", "specific.disorder", target]

X = df3.drop(columns=drop_cols, errors="ignore")
y = df3[target]

print("X shape:", X.shape, "y shape:", y.shape)
print("Class distribution:\n", y.value_counts())


X shape: (387, 1182) y shape: (387,)
Class distribution:
 y_3clases
DEP        199
HEALTHY     95
AUD         93
Name: count, dtype: int64


In [23]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, PowerTransformer, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
import numpy as np

num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X.select_dtypes(exclude=[np.number]).columns.tolist()

numeric_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("power", PowerTransformer(method="yeo-johnson")),
    ("scaler", StandardScaler())
])

categorical_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer([
    ("num", numeric_pipe, num_cols),
    ("cat", categorical_pipe, cat_cols)
], remainder="drop")

clf = LogisticRegression(max_iter=5000, class_weight="balanced")

# 1) ANOVA selection
pipe_anova = Pipeline([
    ("preprocess", preprocess),
    ("var", VarianceThreshold(threshold=0.0)),
    ("select", SelectKBest(score_func=f_classif, k=250)),  # ajusta k si quieres
    ("clf", clf)
])

# 2) PCA extraction
pipe_pca = Pipeline([
    ("preprocess", preprocess),
    ("var", VarianceThreshold(threshold=0.0)),
    ("pca", PCA(n_components=0.95, random_state=42)),
    ("clf", clf)
])

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scores_anova = cross_val_score(pipe_anova, X, y, cv=cv, scoring="f1_macro")
scores_pca   = cross_val_score(pipe_pca,   X, y, cv=cv, scoring="f1_macro")

print("ANOVA macro-F1:", scores_anova.mean(), "+/-", scores_anova.std())
print("PCA   macro-F1:", scores_pca.mean(), "+/-", scores_pca.std())


ANOVA macro-F1: 0.6369419737063138 +/- 0.0473587551087498
PCA   macro-F1: 0.5363662277758023 +/- 0.027355282310720742


In [ ]:
import os
import pandas as pd

path = "/content/EEG.machinelearing_data_BRMH.csv.zip"

print("Exists?", os.path.exists(path))
print("Size (bytes):", os.path.getsize(path) if os.path.exists(path) else None)

df = pd.read_csv(path, compression="zip")

print("Loaded OK ✅")
print("Shape:", df.shape)
print("Columns:", len(df.columns))
print(df.head(3).to_string(index=False))

print("\nMissing values (top 10):")
print(df.isna().sum().sort_values(ascending=False).head(10))



Exists? True
Size (bytes): 4946041
Loaded OK ✅
Shape: (945, 1149)
Columns: 1149
 no. sex  age  eeg.date  education    IQ      main.disorder    specific.disorder  AB.A.delta.a.FP1  AB.A.delta.b.FP2  AB.A.delta.c.F7  AB.A.delta.d.F3  AB.A.delta.e.Fz  AB.A.delta.f.F4  AB.A.delta.g.F8  AB.A.delta.h.T3  AB.A.delta.i.C3  AB.A.delta.j.Cz  AB.A.delta.k.C4  AB.A.delta.l.T4  AB.A.delta.m.T5  AB.A.delta.n.P3  AB.A.delta.o.Pz  AB.A.delta.p.P4  AB.A.delta.q.T6  AB.A.delta.r.O1  AB.A.delta.s.O2  AB.B.theta.a.FP1  AB.B.theta.b.FP2  AB.B.theta.c.F7  AB.B.theta.d.F3  AB.B.theta.e.Fz  AB.B.theta.f.F4  AB.B.theta.g.F8  AB.B.theta.h.T3  AB.B.theta.i.C3  AB.B.theta.j.Cz  AB.B.theta.k.C4  AB.B.theta.l.T4  AB.B.theta.m.T5  AB.B.theta.n.P3  AB.B.theta.o.Pz  AB.B.theta.p.P4  AB.B.theta.q.T6  AB.B.theta.r.O1  AB.B.theta.s.O2  AB.C.alpha.a.FP1  AB.C.alpha.b.FP2  AB.C.alpha.c.F7  AB.C.alpha.d.F3  AB.C.alpha.e.Fz  AB.C.alpha.f.F4  AB.C.alpha.g.F8  AB.C.alpha.h.T3  AB.C.alpha.i.C3  AB.C.alpha.j.Cz  AB.C.alpha.k.C4 

In [ ]:
import numpy as np

# 1) borrar columnas tipo "Unnamed:*" (suelen ser índices guardados)
unnamed = [c for c in df.columns if c.lower().startswith("unnamed")]
print("Dropping Unnamed cols:", len(unnamed))
df = df.drop(columns=unnamed, errors="ignore")

# 2) borrar columnas 100% nulas
all_null = df.columns[df.isna().all()].tolist()
print("Dropping all-null cols:", len(all_null))
df = df.drop(columns=all_null)

print("New shape:", df.shape)


Dropping Unnamed cols: 1
Dropping all-null cols: 0
New shape: (945, 1148)


In [ ]:
summary_low_card = (
    df.nunique(dropna=True)
      .sort_values()
      .reset_index()
)
summary_low_card.columns = ["column", "n_unique"]
summary_low_card = summary_low_card.query("n_unique >= 2 and n_unique <= 20").head(40)

summary_missing = (
    df.isna().sum()
      .sort_values(ascending=False)
      .reset_index()
)
summary_missing.columns = ["column", "n_missing"]
summary_missing = summary_missing.head(30)

summary_dtypes = df.dtypes.value_counts().reset_index()
summary_dtypes.columns = ["dtype", "count"]

print("=== low-cardinality candidates (top 40) ===")
print(summary_low_card.to_string(index=False))

print("\n=== missing columns (top 30) ===")
print(summary_missing.to_string(index=False))

print("\n=== dtypes ===")
print(summary_dtypes.to_string(index=False))


=== low-cardinality candidates (top 40) ===
           column  n_unique
              sex         2
    main.disorder         7
specific.disorder        12
        education        19

=== missing columns (top 30) ===
              column  n_missing
           education         15
                  IQ         13
COH.D.beta.j.Cz.s.O2          0
COH.D.beta.j.Cz.r.O1          0
COH.D.beta.j.Cz.q.T6          0
COH.D.beta.j.Cz.p.P4          0
COH.D.beta.j.Cz.o.Pz          0
COH.D.beta.j.Cz.n.P3          0
COH.D.beta.j.Cz.m.T5          0
COH.D.beta.j.Cz.l.T4          0
COH.D.beta.k.C4.l.T4          0
COH.D.beta.i.C3.s.O2          0
COH.D.beta.i.C3.r.O1          0
COH.D.beta.i.C3.q.T6          0
COH.D.beta.i.C3.p.P4          0
COH.D.beta.i.C3.o.Pz          0
COH.D.beta.i.C3.n.P3          0
COH.D.beta.i.C3.m.T5          0
COH.D.beta.j.Cz.k.C4          0
COH.D.beta.m.T5.p.P4          0
COH.D.beta.m.T5.o.Pz          0
COH.D.beta.m.T5.n.P3          0
COH.D.beta.l.T4.s.O2          0
COH.D.beta.l.T

In [ ]:
# --- Load + quick profiling (Colab) ---
import os
import pandas as pd

path = "/content/EEG.machinelearing_data_BRMH.csv.zip"

print("Exists?", os.path.exists(path))
print("Size (bytes):", os.path.getsize(path) if os.path.exists(path) else None)

df = pd.read_csv(path, compression="zip")

display(df.head())
print("Loaded OK ✅")
print("Shape:", df.shape)
print("Columns:", len(df.columns))

print("\nDtypes count:\n", df.dtypes.value_counts())

print("\nMissing values (top 20):")
print(df.isna().sum().sort_values(ascending=False).head(20))

# --- Define feature groups (AB / COH) + basic target candidates ---
meta_cols = ["no.", "sex", "age", "eeg.date", "education", "IQ", "main.disorder", "specific.disorder"]
meta_cols = [c for c in meta_cols if c in df.columns]

ab_cols  = [c for c in df.columns if c.startswith("AB.")]
coh_cols = [c for c in df.columns if c.startswith("COH.")]
other_cols = [c for c in df.columns if c not in set(meta_cols + ab_cols + coh_cols)]

print("\nMeta cols:", len(meta_cols))
print("AB cols:", len(ab_cols))
print("COH cols:", len(coh_cols))
print("Other cols:", len(other_cols))

# Target options
y_main = "main.disorder" if "main.disorder" in df.columns else None
y_spec = "specific.disorder" if "specific.disorder" in df.columns else None

# Example: binary target for Alcohol Use Disorder (AUD) if specific.disorder exists
if y_spec:
    y_aud = df[y_spec].astype(str).str.strip().str.upper().str.contains("ALCOHOL", na=False).astype(int)
    print("\nAUD label distribution:\n", y_aud.value_counts(dropna=False))

# --- Feature matrix examples ---
# 1) EEG-only features (AB + COH)
X_eeg = df[ab_cols + coh_cols]

# 2) EEG + metadata (optional)
X_eeg_meta = df[[c for c in ["sex","age","education","IQ"] if c in df.columns] + ab_cols + coh_cols]

print("\nX_eeg shape:", X_eeg.shape)
print("X_eeg_meta shape:", X_eeg_meta.shape)


Exists? True
Size (bytes): 4946041


,no.,sex,age,eeg.date,education,IQ,main.disorder,specific.disorder,AB.A.delta.a.FP1,AB.A.delta.b.FP2,...,COH.F.gamma.o.Pz.p.P4,COH.F.gamma.o.Pz.q.T6,COH.F.gamma.o.Pz.r.O1,COH.F.gamma.o.Pz.s.O2,COH.F.gamma.p.P4.q.T6,COH.F.gamma.p.P4.r.O1,COH.F.gamma.p.P4.s.O2,COH.F.gamma.q.T6.r.O1,COH.F.gamma.q.T6.s.O2,COH.F.gamma.r.O1.s.O2
0,1,M,57.0,2012.8.30,NaN,NaN,Addictive disorder,Alcohol use disorder,35.998557,21.717375,...,55.989192,16.739679,23.452271,45.678820,30.167520,16.918761,48.850427,9.422630,34.507082,28.613029
1,2,M,37.0,2012.9.6,6.0,120.0,Addictive disorder,Alcohol use disorder,13.425118,11.002916,...,45.595619,17.510824,26.777368,28.201062,57.108861,32.375401,60.351749,13.900981,57.831848,43.463261
2,3,M,32.0,2012.9.10,16.0,113.0,Addictive disorder,Alcohol use disorder,29.941780,27.544684,...,99.475453,70.654171,39.131547,69.920996,71.063644,38.534505,69.908764,27.180532,64.803155,31.485799
3,4,M,35.0,2012.10.8,18.0,126.0,Addictive disorder,Alcohol use disorder,21.496226,21.846832,...,59.986561,63.822201,36.478254,47.117006,84.658376,24.724096,50.299349,35.319695,79.822944,41.141873
4,5,M,36.0,2012.10.18,16.0,112.0,Addictive disorder,Alcohol use disorder,37.775667,33.607679,...,61.462720,59.166097,51.465531,58.635415,80.685608,62.138436,75.888749,61.003944,87.455509,70.531662


Loaded OK ✅
Shape: (945, 1149)
Columns: 1149

Dtypes count:
 float64    1144
object        4
int64         1
Name: count, dtype: int64

Missing values (top 20):
Unnamed: 122            945
education                15
IQ                       13
COH.D.beta.j.Cz.s.O2      0
COH.D.beta.j.Cz.r.O1      0
COH.D.beta.j.Cz.q.T6      0
COH.D.beta.j.Cz.p.P4      0
COH.D.beta.j.Cz.o.Pz      0
COH.D.beta.j.Cz.n.P3      0
COH.D.beta.j.Cz.m.T5      0
COH.D.beta.j.Cz.l.T4      0
COH.D.beta.k.C4.l.T4      0
COH.D.beta.i.C3.s.O2      0
COH.D.beta.i.C3.r.O1      0
COH.D.beta.i.C3.q.T6      0
COH.D.beta.i.C3.p.P4      0
COH.D.beta.i.C3.o.Pz      0
COH.D.beta.i.C3.n.P3      0
COH.D.beta.i.C3.m.T5      0
COH.D.beta.j.Cz.k.C4      0
dtype: int64

Meta cols: 8
AB cols: 114
COH cols: 1026
Other cols: 1

AUD label distribution:
 specific.disorder
0    852
1     93
Name: count, dtype: int64

X_eeg shape: (945, 1140)
X_eeg_meta shape: (945, 1144)


In [ ]:
import pandas as pd

# --- 1) Conteo de clases (main.disorder y specific.disorder) ---
if "main.disorder" in df.columns:
    main_counts = (
        df["main.disorder"]
        .astype(str).str.strip().str.upper()
        .value_counts(dropna=False)
    )
    print("MAIN.DISORDER - #clases:", main_counts.shape[0])
    print(main_counts.head(30))

if "specific.disorder" in df.columns:
    spec_counts = (
        df["specific.disorder"]
        .astype(str).str.strip().str.upper()
        .value_counts(dropna=False)
    )
    print("\nSPECIFIC.DISORDER - #clases:", spec_counts.shape[0])
    print(spec_counts.head(30))

# --- 2) Si tienes ~90 padecimientos: recorta a Top-K y agrupa el resto en "OTHER" ---
K = 10  # cambia a 20/30 si quieres
col = "specific.disorder"  # o "main.disorder"

s = df[col].astype(str).str.strip().str.upper()

vc = s.value_counts(dropna=False)
topk = vc.head(K).index

s_reduced = s.where(s.isin(topk), other="OTHER")
reduced_counts = s_reduced.value_counts(dropna=False)

print(f"\n[{col}] Recorte Top-{K} + OTHER")
print("Clases después de recorte:", reduced_counts.shape[0])
print(reduced_counts)

# --- 3) Tabla completa de conteos por clase (para exportar si quieres) ---
counts_df = vc.rename_axis(col).reset_index(name="n_samples")
display(counts_df.head(50))  # top 50

# Opcional: guardar a CSV
# counts_df.to_csv(f"/content/{col}_class_counts.csv", index=False)


MAIN.DISORDER - #clases: 7
main.disorder
MOOD DISORDER                         266
ADDICTIVE DISORDER                    186
TRAUMA AND STRESS RELATED DISORDER    128
SCHIZOPHRENIA                         117
ANXIETY DISORDER                      107
HEALTHY CONTROL                        95
OBSESSIVE COMPULSIVE DISORDER          46
Name: count, dtype: int64

SPECIFIC.DISORDER - #clases: 12
specific.disorder
DEPRESSIVE DISORDER               199
SCHIZOPHRENIA                     117
HEALTHY CONTROL                    95
ALCOHOL USE DISORDER               93
BEHAVIORAL ADDICTION DISORDER      93
BIPOLAR DISORDER                   67
PANIC DISORDER                     59
POSTTRAUMATIC STRESS DISORDER      52
SOCIAL ANXIETY DISORDER            48
OBSESSIVE COMPULSITVE DISORDER     46
ACUTE STRESS DISORDER              38
ADJUSTMENT DISORDER                38
Name: count, dtype: int64

[specific.disorder] Recorte Top-10 + OTHER
Clases después de recorte: 11
specific.disorder
DEPRESSIVE DIS

,specific.disorder,n_samples
0,DEPRESSIVE DISORDER,199
1,SCHIZOPHRENIA,117
2,HEALTHY CONTROL,95
3,ALCOHOL USE DISORDER,93
4,BEHAVIORAL ADDICTION DISORDER,93
5,BIPOLAR DISORDER,67
6,PANIC DISORDER,59
7,POSTTRAUMATIC STRESS DISORDER,52
8,SOCIAL ANXIETY DISORDER,48
9,OBSESSIVE COMPULSITVE DISORDER,46
